## 2a. Calculate closed-system degassing path assuming melt composition is the bulk composition (with and without sulfur saturation)

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Remove the # from line below to do this (don't remove the # from this line!).
# pip install VolFe

In [2]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

At a minimum to run a degassing calculations, we need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt [note 1], and temperature at some point along the degassing path:

- Sample is just the name for this analysis.

- Temperature is in 'C.

- Volatile-free melt composition is in wt% oxides [note 2]. All these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see [note 1] around FeOT).

- H2O is all hydrogen in the melt reported as H2O in wt% [note 3]. 

- CO2ppm is all carbon in the melt reported as CO2 in ppm [note 3].

- STppm is all sulfur in the melt reported as S in ppm [note 3].

- Xppm is all "X" in the melt reported as X in ppm [note 3]. X is an unreactive melt species whose identity can be changed - this is explored WHERE.

- Fe3+FeT is the ratio of Fe3+ to FeT in the melt.

[note 1] In this example we specify oxygen fugacity using Fe3+/FeT and Fe in the melt as FeOT as is quite common for melt inclusion and matrix glass analyses - other options are possible, see Example 1a for more information.

[note 2] It does not matter what the non-volatile oxides sum too - they are renormalised to 100 wt% minus the total of the volatiles (i.e., H2O + CO2 + ST + X).

[note 3] The volatile concentrations are absolute for the melt = i.e., the non-volatile melt composition is normalised to 100 wt% minus the volatiles. The volatiles are not added to the oxides and then renormalised.

The following composition is analysis Sari15-04-33 from Brounce et al. (2014), with a temperature chosen as 1200 'C.

In [3]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 4.17, # wt%
           'CO2ppm': 1487., # ppm
           'STppm': 1343.5, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.195}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

For this example we will mostly use the default options in VolFe, which can be found below:

In [ ]:
# print default options in VolFe
print(vf.default_models)

But I'll just highlight the specific options that are important to the degassing calculation and what their default values are:

**bulk_composition = yes** This means the composition (including volatiles and fO2 estimate) represent the bulk composition of the system - there is only melt present and all volatiles are dissolved in the melt.

**starting_P = bulk** This means the calculation will start at Pvsat for the composition given.

**gasssing_style = closed** This means the melt and vapor remain in chemical equilibrium throughout and the bulk composition is constant during the calculation

**gassing_direction = degas** This means pressure will decrease during the calculations and therefore the melt will degas.

The default pressure step is 10 bars, which works most of the time but can be a little slow.
Increasing the step size will speed up the calculation but if the step size is too big it will struggle to solve because it uses the previous step for its initial guesses.

In [ ]:
vf.calc_gassing(my_analysis)

We could run the same calculation from a csv file if we preferred.
We'll use the examples_marianas csv in files and use all the default options again. 
The data in this file are from Brounce et al. (2014, 2016) and Kelley & Cottrell (2012).
Sari15-04-33 is row 48 in that file.

In [ ]:
# Read csv to define melt composition
my_analyses = pd.read_csv("../files/example_marianas.csv") 

vf.calc_gassing(my_analyses,row=48)

Note that we got a warning that the melt was sulfide-saturated during the degassing path. 
Under those conditions, the results for the melt and vapor are metastable with respect to saturation of sulfide.

We can run the same calculation, but instead limit the sulfur content in the melt to sulfide or anhydrite saturation, if it reaches the limit. To do that we change the "sulfur_saturation" option from "no" (sulfur phases cannot saturate) to "yes" (sulfur phases can saturate if the sulfur content is high enough).

In [3]:
# choose the options I want - everything else will use the default options
my_models = [['sulfur_saturation','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

And then we run the degassing calculation with those model options

In [4]:
vf.calc_gassing(my_analysis, models=my_models)

Sari15-04-33
1200.0 3848.9041360851115 2024-04-06 21:15:05.497444
3680.0 : Switching solve species from OCS to OCH
1200.0 2850.0 0.7450624842970717 WARNING: sulfide-saturated 2024-04-06 21:17:04.528945
1200.0 1850.0 0.7382224182382382  2024-04-06 21:18:06.733389
1200.0 850.0 0.5780895094132976  2024-04-06 21:19:13.063948
210.0 : Switching solve species from OCH to OHS
210.0 : Switching solve species from OHS to OCS
solver failed, calculation aborted at P =  210.0 2024-04-06 21:21:54.747340


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,Sari15-04-33,1200.0,3848.904136,47.948689,0.750919,16.760515,9.441557,0.180221,5.927255,11.594191,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:15:05.637055
0,Sari15-04-33,1200.0,3840.0,47.96492463774598,0.75117338647545395,16.766189986132133,9.4447533792846912,0.18028161275410892,5.9292619305795853,11.598117087181009,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:15:06.560625
0,Sari15-04-33,1200.0,3830.0,47.9649314932729,0.75117349383910348,16.766192382488811,9.444754729203666,0.18028163852138501,5.9292627780366587,11.598118744875762,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:15:07.370885
0,Sari15-04-33,1200.0,3820.0,47.964938417181145,0.75117360227366614,16.766194802748231,9.4447560925875536,0.18028166454567973,5.9292636339468032,11.598120419105413,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:15:07.942852
0,Sari15-04-33,1200.0,3810.0,47.964945409855829,0.75117371178517089,16.766197247045028,9.4447574695122256,0.18028169082844103,5.9292644983576182,11.598122109963043,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:15:08.982923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Sari15-04-33,1200.0,260.0,48.011297872530463,0.75189963258295744,16.782399799251635,9.4538847136763877,0.18045591181990983,5.9349944331881446,11.609330327080869,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:20:05.503150
0,Sari15-04-33,1200.0,250.0,48.011465901338056,0.75190226406355265,16.782458533898506,9.4539178001590756,0.18045654337525252,5.9350152043416458,11.609370957141252,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:20:07.373753
0,Sari15-04-33,1200.0,240.0,48.011632400601457,0.75190487159012509,16.782516733891583,9.453950585459836,0.18045716918162991,5.9350357864180552,11.609411217351521,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:20:08.908190
0,Sari15-04-33,1200.0,230.0,48.011797446956848,0.75190745636286493,16.782574426019153,9.4539830846690904,0.18045778952708758,5.9350561888908828,11.609451126242647,...,Ohmoto97,Ohmoto97,ONeill22,Ohmoto97,Ohmoto97,Moussallam19,Holloway92,COS,DensityX,2024-04-06 21:20:10.171601


### in progress ###
Plotting...
